# BioTek EL406

The BioTek EL406 plate washer is controlled by the {class}`~pylabrobot.plate_washing.biotek.el406.BioTekEL406Backend` class. It communicates via FTDI USB serial.

The EL406 has three fluid delivery subsystems — manifold, syringe pump, and peristaltic pump — plus an integrated plate shaker. All operations require a {class}`~pylabrobot.resources.Plate` object to configure plate-specific parameters automatically.

## Setup

In [ ]:
from pylabrobot.plate_washing import PlateWasher
from pylabrobot.plate_washing.biotek.el406 import BioTekEL406Backend
from pylabrobot.resources import Cor_96_wellplate_360ul_Fb

backend = BioTekEL406Backend()  # BioTekEL406Backend(device_id="YOUR_FTDI_ID_HERE")
washer = PlateWasher(
    name="el406",
    size_x=200,
    size_y=200,
    size_z=100,
    backend=backend,
)
await washer.setup()

plate = Cor_96_wellplate_360ul_Fb(name="plate")

## Manifold

The wash manifold is the primary fluid system. Prime the lines before use to fill tubing with buffer.

In [ ]:
await backend.manifold_prime(plate, volume=10000, buffer="A")  # 10 mL

Dispense and aspirate individually, or use `manifold_wash` for repeated dispense-aspirate cycles.

In [ ]:
await backend.manifold_dispense(plate, volume=200, buffer="A")

In [ ]:
await backend.manifold_aspirate(plate)

In [ ]:
await backend.manifold_wash(plate, cycles=3, buffer="A")

`manifold_wash` supports many options: shake/soak between cycles, secondary aspirate, bottom wash, and per-cycle pre-dispense.

In [ ]:
await backend.manifold_wash(
    plate,
    cycles=3,
    buffer="A",
    dispense_volume=300,
    soak_duration=10,
    shake_duration=5,
    shake_intensity="Medium",
)

Run an auto-clean cycle to flush the manifold lines.

In [ ]:
await backend.manifold_auto_clean(plate, buffer="A", duration=60)

## Syringe pump

The syringe pump provides precise low-volume dispensing.

In [ ]:
await backend.syringe_prime(plate, syringe="A", volume=5000, flow_rate=5, refills=2)

In [ ]:
await backend.syringe_dispense(plate, volume=50, syringe="A")

## Peristaltic pump

In [ ]:
await backend.peristaltic_prime(plate, volume=300, flow_rate="High")

In [ ]:
await backend.peristaltic_dispense(plate, volume=100, flow_rate="High")

In [ ]:
await backend.peristaltic_purge(plate, volume=300, flow_rate="High")

## Shaking

In [ ]:
await backend.shake(plate, duration=30, intensity="Medium")

## Batching

Each step command automatically starts and cleans up a batch. To group multiple steps into a single batch (avoiding repeated start/cleanup overhead), use the `batch()` context manager.

In [ ]:
async with backend.batch(plate):
    await backend.manifold_prime(plate, volume=10000, buffer="A")
    await backend.manifold_wash(plate, cycles=3, buffer="A")
    await backend.manifold_aspirate(plate)

## Queries

In [ ]:
await backend.request_serial_number()

In [ ]:
await backend.request_washer_manifold()

In [ ]:
await backend.request_instrument_settings()

## Teardown

In [ ]:
await washer.stop()